<a href="https://colab.research.google.com/github/rono-victor/image-detector/blob/main/Copy_of_yolov8_imgdetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Developing an image detection model using YOLOv8



##Dataset
Camera trap images from the DSAIL-Porini dataset.

There are 7 classes, from the 6 animal species (Impala, Zebra, Warthog, Bushbuck, Waterbuck, Monkey) and one class 'Other' for any other animal in the dataset not among the six species.

The images were manually annotated using makesense.ai and the labels exported as a zip file in YOLO format.

##Model
YOLOm
epochs =



In [ ]:
#install ultralytics package
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 4.0 MB/s eta 0:00:00


In [ ]:
#import the necesssary libraries
import os
import shutil
import random
#tqdm for progress bar
!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [ ]:
#connect drive to the notebook (so that the notebook can read and write to folders on the google drive)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#define paths to train, validation and test folders to store the respective data in the folders
train_path_img = "./yolov8_data2/images/train/"
train_path_label = "./yolov8_data2/labels/train/"
val_path_img = "./yolov8_data2/images/val/"
val_path_label = "./yolov8_data2/labels/val/"
test_path = "./yolov8_data2/test/"

In [ ]:
'''
Define a function to split the dataset into train and test datasets in a 80:20 ratio
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("<<<<<<<< PROCESS STARTED >>>>>>>>")

    #{os.listdir(path)}return a list of filenames prsent in the dataset directory
    #{name[:-4] for name in os.listdir(path)}iterate over the filenames and take a substring of the filename excluding the last 4 characters
    #{set(...)}incase of duplicate filenames in the list, remove them
    files = list(set([name[:-4] for name in os.listdir(path)]))


    #Print out the total number of image files
    print (f"<<<< This folder has a total number of {len(files)} images>>>>")
    #set the random seed to ensure reproducibility
    random.seed(42)
    #randomly shuffle images in the 'files' list
    random.shuffle(files)

    #get the number/length of images and create the test&train sizes based on the ratio defined earlier(80:20)
    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories according to paths defined earlier, if directories exist proceed without raising an error
    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    #Iterate over the list 'files' copying images&labels from main data dir(path) to respective train&label directories
    #
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')


    #print out number of train images created
    print(f"<<<<<<<< Training data created with 80% split {len(files[:train_size])} images >>>>>>>>")

    #copy images from neg_path dir (if exists) to train_path_img
    #neg_path takes care of the negative examples(if provided)
    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"<<<<<<< Total  {len(neg_images)} negative images added to the training data >>>>>>>")

        print(f"<<<<<< TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images >>>>>>")



    # copytin images from the dir 'path' to the validation folder created
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':#skip any file with the name 'classes'
          continue
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    #print out the number of validation data created
    print(f"<<<<<<<< Validation data created with a total of {len(files[train_size:])} images >>>>>>>>")



    #Confirm completion of data spliting
    print("<<<<<<<<<<<<<<< TASK COMPLETED >>>>>>>>>>>>>>>>")

#This is the  'path' directory. Contains data to be split
train_test_split('/content/drive/MyDrive/The_DSAIL/data2/')
#for the negative images
#train_test_split('./data/','./negative_images/')

<<<<<<<< PROCESS STARTED >>>>>>>>
<<<< This folder has a total number of 4305 images>>>>


  0%|          | 0/3444 [00:00<?, ?it/s]

<<<<<<<< Training data created with 80% split 3444 images >>>>>>>>


  0%|          | 0/861 [00:00<?, ?it/s]

<<<<<<<< Validation data created with a total of 861 images >>>>>>>>
<<<<<<<<<<<<<<< TASK COMPLETED >>>>>>>>>>>>>>>>


In [ ]:
# #check number of files in the data directory
# files = os.listdir('/content/drive/MyDrive/The_DSAIL/data2/')
# files = [file for file in files if 'jpg' in file]
# len(files)

In [ ]:
#perfom integrity checks of the ultralytics installation and compute environment setup
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.147 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.8/78.2 GB disk)


## Training

In [ ]:
#specify mode, model, yaml file path
#training parameters
#path to save project and folder name(wildlife)
!yolo task = detect model=yolov8m.pt data=/content/drive/MyDrive/The_DSAIL/yolov8_imgdetector.yaml epochs = 50 imgsz=640 batch=8 project=/content/drive/MyDrive/The_DSAIL/training_results2 name=wildlife

WARNING ⚠️ 'mode' is missing. Valid modes are ('train', 'val', 'predict', 'export', 'track', 'benchmark'). Using default 'mode=train'.
100% 49.7M/49.7M [00:02<00:00, 18.6MB/s]
Ultralytics YOLOv8.0.147 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/The_DSAIL/yolov8_imgdetector.yaml, epochs=50, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/The_DSAIL/training_results2, name=wildlife, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=Fals

## Inferencing

In [ ]:
#make predictions on the test images
#load the trained/finetuned model from the drive dir
#set parameters (confidence level,...)
#specify path to the test images
!yolo task = detect mode= predict model= /content/drive/MyDrive/The_DSAIL/training_results2/wildlife4/weights/best.pt conf=0.3 save=True save_txt = True save_json = True source=/content/drive/MyDrive/The_DSAIL/test_images

Ultralytics YOLOv8.0.147 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25843813 parameters, 0 gradients

image 1/325 /content/drive/MyDrive/The_DSAIL/test_images/snapshot_201910831124667.jpg: 480x640 2 Impalas, 61.7ms
image 2/325 /content/drive/MyDrive/The_DSAIL/test_images/snapshot_201910831124745.jpg: 480x640 1 Impala, 28.2ms
image 3/325 /content/drive/MyDrive/The_DSAIL/test_images/snapshot_201910831124823.jpg: 480x640 1 Impala, 28.3ms
image 4/325 /content/drive/MyDrive/The_DSAIL/test_images/snapshot_2019108311250252.jpg: 480x640 1 Impala, 32.7ms
image 5/325 /content/drive/MyDrive/The_DSAIL/test_images/snapshot_2019108311251231.jpg: 480x640 1 Impala, 28.2ms
image 6/325 /content/drive/MyDrive/The_DSAIL/test_images/snapshot_2019108311252209.jpg: 480x640 1 Impala, 28.2ms
image 7/325 /content/drive/MyDrive/The_DSAIL/test_images/snapshot_2019108311253186.jpg: 480x640 1 Impala, 73.1ms
image 8/325 /content/drive/MyDrive/The_DSAIL/test_imag

In [ ]:
#inference on a video
!yolo task=detect mode=predict model=/content/drive/MyDrive/The_DSAIL/training_results2/wildlife4/weights/best.pt  conf= 0.3  source= /content/drive/MyDrive/The_DSAIL/test_video

Ultralytics YOLOv8.0.147 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 218 layers, 25843813 parameters, 0 gradients

video 1/2 (1/299) /content/drive/MyDrive/The_DSAIL/test_video/AnyConv.com__2023-07-14-18-39-44.mp4: 448x640 2 Impalas, 1220.2ms
video 1/2 (2/299) /content/drive/MyDrive/The_DSAIL/test_video/AnyConv.com__2023-07-14-18-39-44.mp4: 448x640 2 Impalas, 1032.6ms
video 1/2 (3/299) /content/drive/MyDrive/The_DSAIL/test_video/AnyConv.com__2023-07-14-18-39-44.mp4: 448x640 2 Impalas, 1363.5ms
video 1/2 (4/299) /content/drive/MyDrive/The_DSAIL/test_video/AnyConv.com__2023-07-14-18-39-44.mp4: 448x640 2 Impalas, 1800.9ms
video 1/2 (5/299) /content/drive/MyDrive/The_DSAIL/test_video/AnyConv.com__2023-07-14-18-39-44.mp4: 448x640 2 Impalas, 1747.2ms
video 1/2 (6/299) /content/drive/MyDrive/The_DSAIL/test_video/AnyConv.com__2023-07-14-18-39-44.mp4: 448x640 2 Impalas, 1243.2ms
video 1/2 (7/299) /content/drive/MyDrive/The_DSAIL/test_video/AnyConv.com__202

## Copy output from runtime to drive folder

In [ ]:
#Copy the predictions and labels made from colab runtime to google drive folder
!cp -r  runs/detect/predict   /content/drive/MyDrive/The_DSAIL/output50

In [ ]:
!cp -r runs/detect/predict/labels   /content/drive/MyDrive/The_DSAIL/output50/labels

In [ ]:
#video
!cp -r runs/detect/predict  /content/drive/MyDrive/The_DSAIL/output50/video

In [ ]:
!apt-get install -y git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.9).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [ ]:
!git config --global user.name "rono-victor"
!git config --global user.email "ronovictor40@gmail.com"


In [ ]:
!git config --global --list


user.name=rono-victor
user.email=ronovictor40@gmail.com


In [ ]:
!git clone https://github.com/rono-victor/image-detector.git



Cloning into 'image-detector'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.


In [ ]:
%cd /content/image-detector

/content/image-detector


In [ ]:
!git add *

In [ ]:
!git commit -m "A copy of image detector model from colab"

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [ ]:
!cp -r /content/image-detector  /content/drive/MyDrive/colab-git


In [ ]:
!git clone https://github.com/Yuri-Njathi/animal-detector.git


Cloning into 'animal-detector'...
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!git add yolov8_imgdetector.
#!git commit -m "Updated example notebook"
#!git push origin main


fatal: pathspec 'image-detector' did not match any files


In [ ]:
#!git commit -m "first commit"
!git config --global user.email "ronovictor40@gmail.com"
!git config --global user.name "rono-victor"

In [ ]:
!git commit -m "first commit"

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [ ]:
!git push origin main

fatal: could not read Username for 'https://github.com': No such device or address
